In [1]:
load_model = 'vit_b_16_based_model2.pt'

data_dir = '../../../stanford_dogs_new/'

In [2]:
# import numpy as np

from IPython.display import Image

import os
import torch
from torch import nn
from torch.utils.data import DataLoader
import torchvision.models as models
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor

from torchvision.datasets import ImageFolder
from PIL import Image

from torch.autograd import Variable

from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy


## load the vit_b_16 based model

In [3]:
vit_b_16_loaded = models.vit_b_16(pretrained=True)

for param in vit_b_16_loaded.parameters():
    param.requires_grad = True

# vit_b_16_loaded.head = nn.Linear(in_features=768, out_features=120, bias=True)
feature_extractor = nn.Sequential(*list(vit_b_16_loaded.children())[-1:])
# feature_extractor[0]
feature_extractor[0].head = nn.Linear(in_features=768, out_features=120, bias=True)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
vit_b_16_loaded = vit_b_16_loaded.to(device)



vit_b_16_loaded.load_state_dict(torch.load('vit_b_16_based_model2.pt'), strict=False)
vit_b_16_loaded.eval()

Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /home/rka73/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth


VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (linear_1): Linear(in_features=768, out_features=3072, bias=True)
          (act): GELU()
          (dropout_1): Dropout(p=0.0, inplace=False)
          (linear_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout_2): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
 

In [4]:
model = vit_b_16_loaded

In [5]:
# preprocess the image for the model
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
    )
])

path = '../../../stanford_dogs_new/test'

list_subfolders_with_paths = [f.path for f in os.scandir(path) if f.is_dir()]

# gets the breed name from subfolders name like 'n02099429-curly-coated_retriever'
def rename(name):
    return ' '.join(' '.join(name.split('-')[1:]).split('_'))

per_class_accuracy = dict()
total_accuracy = 0

for breed_dir in list_subfolders_with_paths:
    
    breed = rename(breed_dir.split('/')[-1])
    
    images = os.listdir(breed_dir)
    
    matched = False
    matches = 0
    total_images = len(images)

    for image in images:

        input_image = Image.open(breed_dir + '/' + image)


        input_tensor = preprocess(input_image)
        if torch.cuda.is_available():
            input_tensor = Variable(input_tensor.cuda())

        input_batch = input_tensor.unsqueeze(0)
        out = model(input_batch)

        probabilities = torch.nn.functional.softmax(out[0], dim=0)
        # print(probabilities)

        with open("../../stanford_dogs_breeds_classes_final.txt", "r") as f:
            categories = [s.strip() for s in f.readlines()]

        predicted_breeds = []
        top1_prob, top1_catid = torch.topk(probabilities, 1)
        for i in range(top1_prob.size(0)):
            # predicted_breeds.append([categories[top3_catid[i]], top3_prob[i].item()*100])
            predicted_breed = categories[top1_catid[i]]

        # list to be used directly by the application (predicted_breed, probability)
        # print("predicted_breeds are: \n", predicted_breed)
        if breed == predicted_breed:
            matched = True
            matches = matches + 1

        per_class_accuracy[breed] = matches / total_images * 100

per_class_accuracy

{'Pomeranian': 100.0,
 'basset': 66.66666666666666,
 'Japanese spaniel': 94.73684210526315,
 'Bouvier des Flandres': 80.0,
 'German shepherd': 75.0,
 'Brabancon griffon': 93.75,
 'Blenheim spaniel': 100.0,
 'kelpie': 81.25,
 'Shih Tzu': 95.45454545454545,
 'curly coated retriever': 75.0,
 'Lakeland terrier': 80.0,
 'Irish setter': 100.0,
 'German short haired pointer': 87.5,
 'flat coated retriever': 62.5,
 'Irish water spaniel': 86.66666666666667,
 'Dandie Dinmont': 94.44444444444444,
 'clumber': 100.0,
 'Yorkshire terrier': 58.82352941176471,
 'schipperke': 87.5,
 'briard': 81.25,
 'miniature poodle': 50.0,
 'chow': 90.0,
 'Saluki': 85.0,
 'boxer': 81.25,
 'Pekinese': 86.66666666666667,
 'komondor': 100.0,
 'Brittany spaniel': 87.5,
 'black and tan coonhound': 87.5,
 'Appenzeller': 56.25,
 'Eskimo dog': 46.666666666666664,
 'giant schnauzer': 87.5,
 'Old English sheepdog': 100.0,
 'Chihuahua': 68.75,
 'miniature schnauzer': 93.75,
 'redbone': 80.0,
 'wire haired fox terrier': 87.5,
 

In [6]:
total_accuracy = sum(per_class_accuracy.values())/120
total_accuracy

82.42645461908234

### Top 10 Breeds with least accuracy

In [7]:
k = 10
sorted(per_class_accuracy.items(), key=lambda x: x[1])[:k]

[('Eskimo dog', 46.666666666666664),
 ('Border collie', 46.666666666666664),
 ('miniature poodle', 50.0),
 ('Walker hound', 50.0),
 ('standard schnauzer', 50.0),
 ('Siberian husky', 50.0),
 ('malamute', 55.55555555555556),
 ('Appenzeller', 56.25),
 ('Cardigan', 56.25),
 ('collie', 56.25)]

### Top 15 Breeds with most accuracy

In [8]:
k = 15
sorted(per_class_accuracy.items(), key=lambda x: x[1], reverse=True)[:k]

[('Pomeranian', 100.0),
 ('Blenheim spaniel', 100.0),
 ('Irish setter', 100.0),
 ('clumber', 100.0),
 ('komondor', 100.0),
 ('Old English sheepdog', 100.0),
 ('keeshond', 100.0),
 ('Kerry blue terrier', 100.0),
 ('groenendael', 100.0),
 ('Weimaraner', 100.0),
 ('Saint Bernard', 100.0),
 ('vizsla', 100.0),
 ('Shih Tzu', 95.45454545454545),
 ('Leonberg', 95.23809523809523),
 ('Airedale', 95.23809523809523)]